In [21]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import documents as docs
import textcomparisons as tc
import random
import time

def readArticles(path):
    """ Reads df of articles from the given path, and adds a column
    to store the Document-processed article """
    article_df = pd.read_csv(path)
    article_df["doc"] = None
    return article_df

def dict_by_ids(df, ids):
    """ Given a dataframe of articles and a list of article ids, 
    returns a dictionary with ids as keys and Documents as items, 
    computing and storing the Documents back in the df as needed
    """
    doc_dict = {}
    for doc_id in ids:
        row = df["id"] == doc_id
        doc = df.loc[row, "doc"].iloc[0]
        if doc is None:
            doc = docs.Document(df.loc[row, "text"].iloc[0], clean = False)
            df.loc[row, "doc"] = doc
        doc_dict[doc_id] = doc
    return doc_dict

data_folder = "data"
article_files = ["articles2019-05-31_0-7000.csv",
                 "articles2019-05-31_7000-14000.csv",
                 "articles2019-05-31_14000-16654.csv"]
article_df = [readArticles(os.path.join(data_folder, file)) for file in article_files]

article_df = pd.concat(article_df)
article_df = article_df.reset_index()
article_df.head()

,index,id,publisher,scrape_time,on_homepage,url,title,text,fingerprint,event,doc
0,0,0,sfgate.com,2019-05-31T07:45:29-04:00,False,https://www.sfgate.com/news/article/Beto-O-Rou...,"Beto O'Rourke bolstering Iowa staff, field off...","Beto O'Rourke bolstering Iowa staff, field off...","{'wordCounts': {'31293': 2, '38276': 5, '15430...",NaN,None
1,1,1,centurylink.net/news,2019-05-31T11:36:56-04:00,True,https://centurylink.net/news/read/category/new...,Robbins Geller Rudman & Dowd LLP Announces Pro...,TO: ### ALL PERSONS WHO PURCHASED DIPLOMAT PHA...,"{'wordCounts': {'744': 3, '421': 3, '13018': 5...",NaN,None
2,2,2,news.google.com,2019-05-31T15:32:23-04:00,True,https://news.google.com/stories/CAAqOQgKIjNDQk...,NaN,iMoreverified_user ### iMore ### From buying u...,"{'wordCounts': {'710161': 1, '403001': 1, '226...",NaN,None
3,3,3,zerohedge.com,2019-05-31T09:23:14-04:00,True,https://www.zerohedge.com/news/2019-05-31/stun...,Stunned Wall Street Responds To Trump's Shock ...,0 ### SHARES ### Just as the trade war with Ch...,"{'wordCounts': {'1169': 1, '310': 2, '127': 22...",6.0,None
4,4,4,washingtonpost.com,2019-05-31T07:18:53-04:00,False,https://www.washingtonpost.com/opinions/behind...,Behind the Iraq invasi,Former secretary of state Colin Powell in Wash...,"{'wordCounts': {'3000': 2, '17': 3, '7289': 2,...",NaN,None


In [ ]:
sample100 = article_df.loc[random.sample(range(len(article_df)), 1000), "id"]

start = time.time()

ac = tc.ArticleComparisons(thresh_jaccard = .25, thresh_same_sent = .9, thresh_same_doc = .25)
article_dict = dict_by_ids(article_df, sample100)

score_mat = ac.jac_score_mat(article_dict, weighted = False)
axislabels = list(article_dict.keys())
axislabels

print("%s seconds taken" % np.round(time.time() - start, 2))
# takes about ~1min for 100 docs

In [27]:
ac.get_article_clusters()
ac.prop_unique_clusters()

0.9702970297029703